### 환자의 진단과 처방을 한 테이블을 만들기 위한 노트북입니다. 
used table : ADDMISSION | DIAGNOSES_ICD | D_ICD_DIAGNOSES | PROCEDURES_ICD | D_ICD_PROCEDURES

In [256]:
import pandas as pd

In [257]:
BASE_DIR = 'data/'

addmission  = pd.read_csv(BASE_DIR + 'ADMISSIONS.csv')

diagnoses   = pd.read_csv(BASE_DIR + 'DIAGNOSES_ICD.csv')
procedures  = pd.read_csv(BASE_DIR + 'PROCEDURES_ICD.csv')

dict_diagnoses  = pd.read_csv(BASE_DIR + 'D_ICD_DIAGNOSES.csv')
dict_procedures = pd.read_csv(BASE_DIR + 'D_ICD_PROCEDURES.csv')

# addmission_diag.info()
# print(addmission_diag['ICD9_CODE'].duplicated(keep=False).sum())
'''
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ROW_ID      58976 non-null  int64 
 1   SUBJECT_ID  58976 non-null  int64 
 2   DIAGNOSIS   58951 non-null  object
'''
# print(diagnoses['ICD9_CODE'].duplicated(keep=False).sum())
# diagnoses.info()
'''
#   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ROW_ID      651047 non-null  int64  
 1   SUBJECT_ID  651047 non-null  int64  
 2   HADM_ID     651047 non-null  int64  
 3   SEQ_NUM     651000 non-null  float64
 4   ICD9_CODE   651000 non-null  object 
'''
# procedures.info()
# print(procedures['ICD9_CODE'].duplicated(keep=False).sum())

'''
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   ROW_ID      240095 non-null  int64
 1   SUBJECT_ID  240095 non-null  int64
 2   HADM_ID     240095 non-null  int64
 3   SEQ_NUM     240095 non-null  int64
 4   ICD9_CODE   240095 non-null  int64
'''
# dict_diagnoses.info()
# print(dict_diagnoses['ICD9_CODE'].duplicated(keep=False).sum())
'''
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ROW_ID       14567 non-null  int64 
 1   ICD9_CODE    14567 non-null  object
 2   SHORT_TITLE  14567 non-null  object
 3   LONG_TITLE   14567 non-null  object
'''
# dict_procedures.info()
# print(dict_procedures['ICD9_CODE'].duplicated(keep=False).sum())
'''
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ROW_ID       3882 non-null   int64 
 1   ICD9_CODE    3882 non-null   int64 
 2   SHORT_TITLE  3882 non-null   object
 3   LONG_TITLE   3882 non-null   object
'''

'\n #   Column       Non-Null Count  Dtype \n---  ------       --------------  ----- \n 0   ROW_ID       3882 non-null   int64 \n 1   ICD9_CODE    3882 non-null   int64 \n 2   SHORT_TITLE  3882 non-null   object\n 3   LONG_TITLE   3882 non-null   object\n'

In [258]:
addmission_diag = addmission[['SUBJECT_ID','HADM_ID', 'DIAGNOSIS']].dropna()
addmission_diag = addmission_diag[addmission_diag['DIAGNOSIS'] != 'NEWBORN']
# diag_ICD = pd.merge(addmission_diag['SUBJECT_ID'], diagnoses[['SUBJECT_ID','ICD9_CODE']], how='outer', on='SUBJECT_ID')

In [259]:
# patient_diag_ICD = pd.merge(diag_ICD, dict_diagnoses.drop('ROW_ID',axis=1), how='inner', on='ICD9_CODE') # 둘 다 있는 경우만
# patient_diag_ICD = patient_diag_ICD.drop_duplicates()
# patient_diag_ICD = patient_diag_ICD.set_index('SUBJECT_ID')
# patient_diag_ICD.sort_values('SUBJECT_ID')

In [260]:
diagnoses_ICD = diagnoses.drop('ROW_ID', axis=1)
diagnoses_ICD = diagnoses_ICD.rename(columns={'ICD9_CODE': 'diag_ICD'})

procedures_ICD = procedures.drop('ROW_ID', axis=1)
procedures_ICD = procedures_ICD.rename(columns={'ICD9_CODE': 'proc_ICD'})

In [261]:
merged_df = pd.merge(diagnoses_ICD, procedures_ICD, 
                     on=['SUBJECT_ID', 'HADM_ID', 'SEQ_NUM'], how='left')

merged_df = pd.merge(merged_df , addmission_diag, 
                     on=['SUBJECT_ID', 'HADM_ID'], how='left')
merged_df = merged_df.dropna()
merged_df

,SUBJECT_ID,HADM_ID,SEQ_NUM,diag_ICD,proc_ICD,DIAGNOSIS
0,109,172335,1.0,40301,5498.0,LEG PAIN
14,109,173633,1.0,40301,3995.0,ABDOMINAL PAIN
15,109,173633,2.0,5856,3893.0,ABDOMINAL PAIN
16,109,173633,3.0,58381,9904.0,ABDOMINAL PAIN
28,112,174105,1.0,53100,4443.0,GASTROINTESTINAL BLEED
...,...,...,...,...,...,...
651032,97497,168949,1.0,0529,9969.0,ABDOMINAL PAIN
651033,97497,168949,2.0,4162,9929.0,ABDOMINAL PAIN
651034,97497,168949,3.0,20192,9925.0,ABDOMINAL PAIN
651041,97503,188195,1.0,7842,260.0,LUDWIGS ANGINA


In [262]:
merged_df = pd.merge(merged_df, 
                    dict_diagnoses[['ICD9_CODE','SHORT_TITLE','LONG_TITLE']], 
                    how='inner', 
                    left_on='diag_ICD',
                    right_on='ICD9_CODE')
merged_df = merged_df.rename(columns={'SHORT_TITLE': 'diag_SHORT_TITLE',
                                      'LONG_TITLE': 'diag_LONG_TITLE',})

merged_df = merged_df.drop('ICD9_CODE', axis=1)

merged_df = pd.merge(merged_df, 
                    dict_procedures[['ICD9_CODE','SHORT_TITLE','LONG_TITLE']], 
                    how='inner', 
                    left_on='proc_ICD',
                    right_on='ICD9_CODE')


merged_df = merged_df.rename(columns={'SHORT_TITLE': 'proc_SHORT_TITLE',
                                      'LONG_TITLE': 'proc_LONG_TITLE',})
merged_df = merged_df.drop('ICD9_CODE', axis=1)
merged_df = merged_df.sort_values('SUBJECT_ID')
merged_df = merged_df.set_index('SUBJECT_ID')
merged_df[['SEQ_NUM','proc_ICD']] = merged_df[['SEQ_NUM','proc_ICD']].astype('int32')
merged_df

,HADM_ID,SEQ_NUM,diag_ICD,proc_ICD,DIAGNOSIS,diag_SHORT_TITLE,diag_LONG_TITLE,proc_SHORT_TITLE,proc_LONG_TITLE
SUBJECT_ID,,,,,,,,,
3,145834,2,78559,9962,HYPOTENSION,Shock w/o trauma NEC,Other shock without mention of trauma,Heart countershock NEC,Other electric countershock of heart
3,145834,5,41071,3893,HYPOTENSION,"Subendo infarct, initial","Subendocardial infarction, initial episode of ...",Venous cath NEC,"Venous catheterization, not elsewhere classified"
3,145834,3,5849,8964,HYPOTENSION,Acute kidney failure NOS,"Acute kidney failure, unspecified",Pulmon art wedge monitor,Pulmonary artery wedge monitoring
3,145834,6,4280,966,HYPOTENSION,CHF NOS,"Congestive heart failure, unspecified",Entral infus nutrit sub,Enteral infusion of concentrated nutritional s...
3,145834,1,0389,9604,HYPOTENSION,Septicemia NOS,Unspecified septicemia,Insert endotracheal tube,Insertion of endotracheal tube
...,...,...,...,...,...,...,...,...,...
99999,113369,1,75612,8108,SPONDYLOLISTHESIS/SDA,Spondylolisthesis,Spondylolisthesis,Lumb/lmbsac fus ant/post,Lumbar and lumbosacral fusion of the anterior ...
99999,113369,5,4139,8451,SPONDYLOLISTHESIS/SDA,Angina pectoris NEC/NOS,Other and unspecified angina pectoris,Ins spinal fusion device,Insertion of interbody spinal fusion device
99999,113369,2,7861,8051,SPONDYLOLISTHESIS/SDA,Stridor,Stridor,Excision intervert disc,Excision of intervertebral disc


In [263]:
def combine_strings(row):
    return f"{row['diag_SHORT_TITLE']} can be solved by {row['proc_SHORT_TITLE']} detailed description is {row['proc_LONG_TITLE']}."
merged_df['solution'] = merged_df.apply(combine_strings, axis=1)

merged_df

,HADM_ID,SEQ_NUM,diag_ICD,proc_ICD,DIAGNOSIS,diag_SHORT_TITLE,diag_LONG_TITLE,proc_SHORT_TITLE,proc_LONG_TITLE,solution
SUBJECT_ID,,,,,,,,,,
3,145834,2,78559,9962,HYPOTENSION,Shock w/o trauma NEC,Other shock without mention of trauma,Heart countershock NEC,Other electric countershock of heart,Shock w/o trauma NEC can be solved by Heart co...
3,145834,5,41071,3893,HYPOTENSION,"Subendo infarct, initial","Subendocardial infarction, initial episode of ...",Venous cath NEC,"Venous catheterization, not elsewhere classified","Subendo infarct, initial can be solved by Veno..."
3,145834,3,5849,8964,HYPOTENSION,Acute kidney failure NOS,"Acute kidney failure, unspecified",Pulmon art wedge monitor,Pulmonary artery wedge monitoring,Acute kidney failure NOS can be solved by Pulm...
3,145834,6,4280,966,HYPOTENSION,CHF NOS,"Congestive heart failure, unspecified",Entral infus nutrit sub,Enteral infusion of concentrated nutritional s...,CHF NOS can be solved by Entral infus nutrit s...
3,145834,1,0389,9604,HYPOTENSION,Septicemia NOS,Unspecified septicemia,Insert endotracheal tube,Insertion of endotracheal tube,Septicemia NOS can be solved by Insert endotra...
...,...,...,...,...,...,...,...,...,...,...
99999,113369,1,75612,8108,SPONDYLOLISTHESIS/SDA,Spondylolisthesis,Spondylolisthesis,Lumb/lmbsac fus ant/post,Lumbar and lumbosacral fusion of the anterior ...,Spondylolisthesis can be solved by Lumb/lmbsac...
99999,113369,5,4139,8451,SPONDYLOLISTHESIS/SDA,Angina pectoris NEC/NOS,Other and unspecified angina pectoris,Ins spinal fusion device,Insertion of interbody spinal fusion device,Angina pectoris NEC/NOS can be solved by Ins s...
99999,113369,2,7861,8051,SPONDYLOLISTHESIS/SDA,Stridor,Stridor,Excision intervert disc,Excision of intervertebral disc,Stridor can be solved by Excision intervert di...


In [264]:
merged_df.to_csv(BASE_DIR+'PAIENT_DIAG_PROC.csv')

In [267]:
# # 파일 경로 설정
# file_path = "diagnosis_values.txt"

# # 'DIAGNOSIS' 열의 고유한(unique) 값만 추출
# unique_values = addmission_diag['DIAGNOSIS'].unique()
# unique_values.sort()
# # 고유한 값들을 txt 파일로 작성
# with open(file_path, 'w') as file:
#     for value in unique_values:
#         # NaN 값인 경우에는 스킵
#         if pd.notna(value):
#             file.write(str(value) + '\n')
